In [136]:
import IPython.display as ipd
import numpy as np
from keras.layers import Conv1D, Dense, Activation, Input, Reshape, Flatten, MaxPooling1D, BatchNormalization, Dropout
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
import sys, random


In [2]:
sys.path.append("data/")

In [3]:
from load_data import load_GAN_Y, load_Discr_Y

In [89]:
gan_X = np.load("data/audio1_gan_X.npy")
discr_X_true = np.load("data/audio1_disc_X.npy")


In [90]:
gan_X = gan_X.reshape([gan_X.shape[0], gan_X.shape[1], 1])

In [91]:
shape = 26460

def create_generator(nodes):
    model = Sequential()
    
    model.add(Conv1D(nodes,(10,), strides=1, input_shape=(shape,1), padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(1, (3,), padding='same', activation='tanh'))
    model.add(Reshape([shape,]))
    
    return model



def create_discriminator(nodes):
    model = Sequential()
    
    model.add(Dense(nodes, input_shape=(shape,)))
    model.add(LeakyReLU())
    
    model.add(Dense(nodes, input_shape=(shape,)))
    model.add(LeakyReLU())
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

def create_models(gen_nodes=128, discr_nodes=200):
    
    gen = create_generator(gen_nodes)
    discr = create_discriminator(discr_nodes)

    sound = Input(shape=(shape,1,))
    out_gen = gen(sound)
    pred = discr(out_gen)
    
    model = Model(sound, pred)
    
    model.compile(optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    return gen, discr, model
    

In [85]:
gen, discr, model = create_models()

In [86]:
gen.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_34 (Conv1D)           (None, 26460, 128)        1408      
_________________________________________________________________
leaky_re_lu_43 (LeakyReLU)   (None, 26460, 128)        0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 26460, 1)          385       
_________________________________________________________________
reshape_22 (Reshape)         (None, 26460)             0         
Total params: 1,793
Trainable params: 1,793
Non-trainable params: 0
_________________________________________________________________


In [87]:
discr.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 200)               5292200   
_________________________________________________________________
leaky_re_lu_44 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 200)               40200     
_________________________________________________________________
leaky_re_lu_45 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 201       
Total params: 5,332,601
Trainable params: 5,332,601
Non-trainable params: 0
_________________________________________________________________


In [92]:
def train(GAN, Discr, model, GAN_X, Discr_X_true, epochs=1):
    
    length = GAN_X.shape[0]
    
    Discr_Y_true = load_Discr_Y(length, False)
    Discr_Y_false = load_Discr_Y(length, True)
    Discr_Y = np.append(Discr_Y_true, Discr_Y_false).reshape([length*2, 1])

    GAN_Y = load_GAN_Y(length)
    
    train_discr = True
    train_gen = True
    
    
    for i in range(epochs):
    
        
        if train_discr:
            
            Discr.trainable=True

            Discr.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['accuracy'])

            print("Training Discriminator")
            # build discriminator training set
            Discr_X_false = GAN.predict(GAN_X)

            Discr_X = np.append(Discr_X_true, Discr_X_false, axis=0)

            output = discr.fit(Discr_X, Discr_Y, shuffle="true")
            
            print(output.history["acc"])
            if output.history["acc"][0] < .8:
                train_gen = False
            else:
                train_gen = True
        
        if train_gen:
        
        
            Discr.trainable=False

            model.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['accuracy'])


            print("Training Generator")

            output = model.fit(GAN_X, GAN_Y)
            
            if output.history["acc"][0] < .8:
                train_discr = False
            else:
                train_discr = True

In [140]:
train(gen, discr, model, gan_X, discr_X_true, 7)

Training Discriminator


KeyboardInterrupt: 

In [144]:
x=11
ipd.Audio(gen.predict(gan_X[x:x+1]), rate=22050)


In [145]:
ipd.Audio(discr_X_true[x:x+1].reshape([-1, 26460]), rate=22050)

In [146]:
ipd.Audio(gan_X[x:x+1].reshape([26460]), rate=22050)